# Working with Schedule

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../jobs/configuration.ipynb) 
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create schedule from Python SDK
- Enable/Disable schedule from Python SDK
- Update schedule's trigger from Python SDK

**Motivations** - This notebook covers the main scenario that user management schedule for job in SDK v2.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [4]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [5]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Shared token cache unavailable
	VisualStudioCodeCredential: Azure Active Directory error '(invalid_grant) AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2022-01-12T00:05:41.8429245Z and was inactive for 90.00:00:00.
Trace ID: 2ae53689-b9d1-46d8-aa1a-7ff643e32400
Correlation ID: 1afe0522-c590-43f7-9d7a-9ec6751acce7
Timestamp: 2022-08-02 09:02:15Z'
Content: {"error":"invalid_grant","error_description":"AADSTS700082: The refresh token has expired due to inactivity. The token was 

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In [6]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"

    # get a handle to the workspace
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

Found the config file in: .\config.json


MLClient(credential=<azure.identity._credentials.browser.InteractiveBrowserCredential object at 0x000001D335B22490>,
         subscription_id=96aede12-2f73-41cb-b983-6d11a904839b,
         resource_group_name=sdk,
         workspace_name=sdk-master)


## 1.4 Create a pipeline job with settings

In [7]:
parent_dir = "../jobs/pipelines/1a_pipeline_with_components_from_yaml"
train_model = load_component(path=parent_dir + "/train_model.yml")
score_data = load_component(path=parent_dir + "/score_data.yml")
eval_model = load_component(path=parent_dir + "/eval_model.yml")

# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }


pipeline_job = pipeline_with_components_from_yaml(
    training_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    test_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"

## 2.1 Create schedule
### 2.1.1 Create schedule with with recurrence pattern

In [8]:
from azure.ai.ml.entities import JobSchedule,CronTrigger,RecurrenceTrigger # put it to upper once SDK work is ready
from datetime import datetime
from dateutil import tz
from azure.ai.ml.constants import TimeZone, RecurrencePattern

schedule_name="simple_sdk_create_schedule_recurrence"

schedule_start_time = datetime.now(tz=tz.tzutc())
recurrence_trigger = ScheduleRecurrenceTrigger(
    frequency="day",
    interval=1,
    schedule=RecurrencePattern(hours=10, minutes=[0, 1]),
    start_time=schedule_start_time,
    time_zone=TimeZone.UTC,
    create_job=pipeline_job
)

job_schedule = JobSchedule(
    name=schedule_name,
    trigger=recurrence_trigger,
    create_job=pipeline_job
)


# ml_client.schedules.begin_create_or_update(schedule=job_schedule)

ImportError: cannot import name 'RecurrencePattern' from 'azure.ai.ml.constants' (d:\anconda\envs\sdk-preview\lib\site-packages\azure\ai\ml\constants.py)

In [1]:
from azure.ai.ml._version import VERSION
print(VERSION)

0.0.68331024


### 2.1.2 Create schedule with with cron expression

In [9]:
from azure.ai.ml.entities import JobSchedule,CronTrigger,RecurrenceTrigger # put it to upper once SDK work is ready

schedule_name="simple_sdk_create_schedule_cron"

cron_trigger = ScheduleCronTrigger(
    expression="15 10 * * *",
    start_time="2022-07-10T10:00:00", # start time 
    time_zone= "Eastern Standard Time" # time zone of expression
)

job_schedule = JobSchedule(
    name=schedule_name,
    trigger=cron_trigger,
    create_job=pipeline_job
)


ml_client.schedules.begin_create_or_update(schedule=job_schedule)

JobSchedule({'name': 'simple_sdk_create_schedule_cron', 'description': None, 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': 'd:\\azureml-examples\\sdk\\schedules', 'creation_context': <azure.ai.ml._restclient.v2022_06_01_preview.models._models_py3.SystemData object at 0x000001D337B675E0>, 'serialize': <msrest.serialization.Serializer object at 0x000001D337B817C0>, 'trigger': <azure.ai.ml.entities._schedule.trigger.ScheduleCronTrigger object at 0x000001D337B5AA30>, 'display_name': None, 'create_job': PipelineJob({'inputs': {'training_input': <azure.ai.ml.entities._job.pipeline._io.PipelineInput object at 0x000001D337B81550>, 'test_input': <azure.ai.ml.entities._job.pipeline._io.PipelineInput object at 0x000001D337B81580>, 'training_max_epochs': <azure.ai.ml.entities._job.pipeline._io.PipelineInput object at 0x000001D337B815B0>, 'training_learning_rate': <azure.ai.ml.entities._job.pipeline._io.PipelineInput object at 0x000001D337B815E0>, 'learning_r

## 2.2 Disable the schedule

In [ ]:
ml_client.schedules.begin_disable(name=schedule_name)

## 2.3 Check the detail of the schedule

In [ ]:
ml_client.schedules.get(name=schedule_name)

## 2.4 List schedules in a workspace

In [ ]:
ml_client.schedules.list()

## 2.5 Enable a schedule

In [ ]:
ml_client.schedules.begin_enable(name=schedule_name)

## 2.6 Delete the schedule

In [ ]:
# Only disabled schedules can be deleted
ml_client.schedules.begin_disable(name=schedule_name)
ml_client.schedules.delete(name=schedule_name)